# UAS Logika Fuzzy: Implementasi Metode Mamdani dan Sugeno
**Kasus:** Sistem Kendali Kecepatan Kipas AC Otomatis  
**Nama:** Ahmad Dunaza Indrawangsa  
**NIM:** G.231.22.0117

---
## Deskripsi Kasus
Sistem ini menentukan kecepatan kipas AC berdasarkan dua variabel input:
1. **Suhu Ruangan:** Dingin, Nyaman, Panas.
2. **Jumlah Orang:** Sedikit, Sedang, Banyak.

Output yang dihasilkan adalah **Kecepatan Kipas (0-100%)**.

In [1]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 25.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# ==========================================
# 1. DEFINISI SEMESTA & FUNGSI KEANGGOTAAN
# ==========================================
x_suhu = np.arange(0, 41, 1)        # 0-40 derajat Celsius
x_orang = np.arange(0, 21, 1)       # 0-20 orang
x_kipas = np.arange(0, 101, 1)      # 0-100 % kecepatan

# Membership Suhu
suhu_dingin = fuzz.trimf(x_suhu, [0, 0, 20])
suhu_nyaman = fuzz.trimf(x_suhu, [15, 25, 35])
suhu_panas = fuzz.trimf(x_suhu, [30, 40, 40])

# Membership Jumlah Orang
orang_sedikit = fuzz.trimf(x_orang, [0, 0, 10])
orang_sedang = fuzz.trimf(x_orang, [5, 10, 15])
orang_banyak = fuzz.trimf(x_orang, [10, 20, 20])

# Membership Kipas (Untuk Mamdani)
kipas_lambat = fuzz.trimf(x_kipas, [0, 0, 50])
kipas_sedang = fuzz.trimf(x_kipas, [25, 50, 75])
kipas_cepat = fuzz.trimf(x_kipas, [50, 100, 100])

# ==========================================
# 2. IMPLEMENTASI KEDUA METODE
# ==========================================
def hitung_ac(suhu_val, orang_val):
    # --- Fuzzifikasi ---
    u_dingin = fuzz.interp_membership(x_suhu, suhu_dingin, suhu_val)
    u_nyaman = fuzz.interp_membership(x_suhu, suhu_nyaman, suhu_val)
    u_panas = fuzz.interp_membership(x_suhu, suhu_panas, suhu_val)

    u_sedikit = fuzz.interp_membership(x_orang, orang_sedikit, orang_val)
    u_sedang = fuzz.interp_membership(x_orang, orang_sedang, orang_val)
    u_banyak = fuzz.interp_membership(x_orang, orang_banyak, orang_val)

    # --- Aturan (Rules) ---
    # R1: Jika Suhu Dingin AND Orang Sedikit -> Kipas Lambat
    a1 = np.fmin(u_dingin, u_sedikit)

    # R2: Jika Suhu Nyaman -> Kipas Sedang
    a2 = u_nyaman

    # R3: Jika Suhu Panas OR Orang Banyak -> Kipas Cepat
    a3 = np.fmax(u_panas, u_banyak)

    # --- METODE MAMDANI (Area-based) ---
    imp1 = np.fmin(a1, kipas_lambat)
    imp2 = np.fmin(a2, kipas_sedang)
    imp3 = np.fmin(a3, kipas_cepat)
    agregasi = np.fmax(imp1, np.fmax(imp2, imp3))
    hasil_mamdani = fuzz.defuzz(x_kipas, agregasi, 'centroid')

    # --- METODE SUGENO (Constant-based) ---
    # Konstanta: Lambat=20, Sedang=50, Cepat=90
    z1, z2, z3 = 20, 50, 90
    w = np.array([a1, a2, a3])
    z = np.array([z1, z2, z3])
    if np.sum(w) > 0:
        hasil_sugeno = np.sum(w * z) / np.sum(w)
    else:
        hasil_sugeno = 0

    # --- Visualisasi ---
    print(f"HASIL ANALISIS:")
    print(f"Mamdani (Centroid): {hasil_mamdani:.2f}%")
    print(f"Sugeno (W. Average): {hasil_sugeno:.2f}%")

    plt.figure(figsize=(10, 4))
    plt.plot(x_kipas, kipas_lambat, 'b', label='Lambat')
    plt.plot(x_kipas, kipas_sedang, 'g', label='Sedang')
    plt.plot(x_kipas, kipas_cepat, 'r', label='Cepat')
    plt.fill_between(x_kipas, 0, agregasi, color='orange', alpha=0.3, label='Area Mamdani')
    plt.axvline(hasil_mamdani, color='blue', linestyle='--', label='Mamdani Result')
    plt.axvline(hasil_sugeno, color='red', linestyle=':', label='Sugeno Result')
    plt.title(f"Perbandingan Output (Suhu: {suhu_val}, Orang: {orang_val})")
    plt.legend()
    plt.show()

# --- 3. WIDGET INTERAKTIF ---
interact(
    hitung_ac,
    suhu_val=FloatSlider(min=0, max=40, step=1, value=30, description='Suhu C'),
    orang_val=FloatSlider(min=0, max=20, step=1, value=5, description='Jml Orang')
)

interactive(children=(FloatSlider(value=30.0, description='Suhu C', max=40.0, step=1.0), FloatSlider(value=5.0…

<function __main__.hitung_ac(suhu_val, orang_val)>

## Analisis Perbandingan

Berdasarkan hasil eksekusi program di atas, kita dapat menyimpulkan perbedaan antara kedua metode:

1. **Metode Mamdani:**
   - Menghasilkan nilai output berdasarkan penggabungan area fuzzy.
   - Hasil cenderung lebih "halus" dan intuitif secara linguistik.
   - Menggunakan metode **Centroid** untuk mencari titik berat area hasil agregasi.

2. **Metode Sugeno:**
   - Menghasilkan nilai output berdasarkan rata-rata terbobot dari konstanta yang ditentukan.
   - Sangat efisien secara komputasi dan memberikan hasil yang lebih presisi untuk sistem kontrol.
   - Cocok digunakan jika output sistem bersifat diskrit/konstan.